In [1]:
%load_ext autoreload
%autoreload 2
%xmode Plain

Exception reporting mode: Plain


In [2]:
import pandas as pd
import plotly
import plotly.express as px 
import dtale
import numpy as np
import random
from bazaraki import utils
from tqdm import tqdm
import swifter
from pathlib import Path
from datetime import datetime

In [3]:
tqdm.pandas()
pd.set_option('display.max_rows', 100)  # Disable row limit
pd.set_option('display.max_columns', 60)  # Disable column limit
pd.set_option('display.width', 20)  # Disable line width limit
pd.set_option('display.max_colwidth', 100)  # Disable column width limit
pd.set_option('display.precision', 2)  
pd.set_option('display.float_format', '{:.4f}'.format)  
np.random.seed(42)
random.seed(42)
pd.options.plotting.backend = "plotly"
plotly.io.renderers.default = "notebook_connected"
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
rentdf = utils.read_dfs("output/*.parquet")

Reading output/2024-12-12 18:34:25 real-estate-to-rent_real-estate-for-sale.parquet
Reading output/2024-12-14 11:44:22 real-estate-to-rent_real-estate-for-sale.parquet
Total: 35559 valid: 34669 read: 34669 new: 745 deleted: 890 undeleted: 0
Reading output/2024-12-15 18:00:14 real-estate-to-rent_real-estate-for-sale.parquet
Total: 35734 valid: 34398 read: 34398 new: 175 deleted: 490 undeleted: 44
Reading output/2024-12-16 23:13:52 real-estate-to-rent_real-estate-for-sale.parquet
Total: 36281 valid: 34510 read: 34510 new: 547 deleted: 648 undeleted: 213
Reading output/2024-12-17 21:31:02 real-estate-to-rent_real-estate-for-sale.parquet
Total: 37085 valid: 34681 read: 34681 new: 804 deleted: 817 undeleted: 184
Reading output/2024-12-18 23:01:10 real-estate-to-rent_real-estate-for-sale.parquet
Total: 37577 valid: 34809 read: 34809 new: 492 deleted: 468 undeleted: 104
Reading output/2024-12-19 22:09:26 real-estate-to-rent_real-estate-for-sale.parquet
Total: 38047 valid: 33723 read: 33723 ne

AttributeError: 'DataFrame' object has no attribute 'ad_id'

In [ ]:
rentdf.iloc[1].T

IndexError: single positional indexer is out-of-bounds

In [ ]:
rentdf = utils.enrich(rentdf)
rentdf = utils.filter_in(rentdf, "cat1 == 'Apartments, flats to rent'")
rentdf.head()


removing 128979/166899 rows


,url,title,price,original_price,price_per_sqm,location,posted,reference_number,views,lat,lng,sold,cat0,cat1,Property area,Pets,Type,Parking,Plot area,Furnishing,Included,Online viewing,Air conditioning,Energy Efficiency,Bedrooms,Bathrooms,images,description,Floor,Postal code,Construction year,Reference number,Condition,Square meter price,Minimum stay,Land type,Plot Type,Parcel number,Planning zone,Registration number,Share,Density,Coverage,Registration block,Area,Pick a point,posted_dt,delete_date,city,district
ad_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5508233,https://www.bazaraki.com/adv/5508233_2-bedroom-apartment-to-rent/,2-bedroom apartment to rent,1150.0000,NaN,11.5000,"Paphos, Chlorakas",Yesterday 21:02,NaN,385,34.7904,32.4153,False,Cyprus real estate to rent,"Apartments, flats to rent",100.0000,Allowed,Apartment,Uncovered,NaN,Fully Furnished,"Pool, Balcony",No,"Full, all rooms",A,2,1,"[https://cdn1.bazaraki.com/media/cache1/55/3c/553c55844092d64dcf6b81b88fdeaa83.webp, https://cdn...",This beautiful 2 bedroom apartment is located on a hillside in the area of Paphos which has uns...,2nd,8220.0000,2008,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,NaN,None,2024-12-11T21:02:00,2025-01-11,Paphos,Chlorakas
5395481,https://www.bazaraki.com/adv/5395481_2-bedroom-apartment-to-rent/,2-bedroom apartment to rent,1100.0000,NaN,12.2222,"Larnaca, Livadia Larnakas",07.12.2024 11:07,NaN,26,34.9548,33.6378,False,Cyprus real estate to rent,"Apartments, flats to rent",90.0000,Not allowed,Apartment,Covered,NaN,Fully Furnished,"Pool, Elevator, Balcony",Yes,"Full, all rooms",A,2,2,"[https://cdn1.bazaraki.com/media/cache1/26/c8/26c8d90c413acdce05615bf5b068e203.webp, https://cdn...","This is a 2 bedroom, 2 bathroom flat in a beautifully refurbished complex. The flat has been new...",1st,7060.0000,2015,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,NaN,None,2024-12-07T11:07:00,2025-01-20,Larnaca,Livadia Larnakas
5562261,https://www.bazaraki.com/adv/5562261_1-bedroom-apartment-to-rent/,1-bedroom apartment to rent,720.0000,NaN,12.0000,"Nicosia, Nicosia - Ag. Antonios",07.12.2024 12:34,NaN,45,35.1725,33.3650,False,Cyprus real estate to rent,"Apartments, flats to rent",60.0000,Not allowed,Apartment,Covered,NaN,Unfurnished,None,None,"Full, all rooms",B,1,1,"[https://cdn1.bazaraki.com/media/cache1/93/04/93042a9a5112ec854815cc86678202cd.webp, https://cdn...",Rent this beautiful Flat through www.rentonline.today and save 30% on your first monthly rent!. ...,1st,NaN,2015,rentonline-107,None,None,None,None,None,None,None,NaN,None,None,None,NaN,NaN,None,2024-12-07T12:34:00,2025-02-10,Nicosia,Nicosia - Ag. Antonios
5218026,https://www.bazaraki.com/adv/5218026_2-bedroom-apartment-to-rent/,2-bedroom apartment to rent,2900.0000,NaN,38.6667,"Paphos, Aphrodite hills Kouklia",06.12.2024 20:09,NaN,30,NaN,NaN,False,Cyprus real estate to rent,"Apartments, flats to rent",75.0000,None,Apartment,None,NaN,None,None,None,None,N/A,2,2,"[https://cdn1.bazaraki.com/media/cache1/73/2e/732eee6b5e83cd1c8913ca5a10f30a76.webp, https://cdn...",AL21 is a beautiful 2-bedroom Premium Serviced Apartment located on the Alexander Heights comple...,None,NaN,None,CN8293,None,None,None,None,None,None,None,NaN,None,None,None,NaN,NaN,None,2024-12-06T20:09:00,NaN,Paphos,Aphrodite hills Kouklia
5525766,https://www.bazaraki.com/adv/5525766_1-bedroom-apartment-to-rent/,1-bedroom apartment to rent,1200.0000,NaN,18.4615,"Limassol, Polemidia Kato",08.12.2024 19:03,NaN,37,NaN,NaN,False,Cyprus real estate to rent,"Apartments, flats to rent",65.0000,Allowed,Apartment,Covered,NaN,Fully Furnished,"Elevator, Balcony, Storage room",Yes,"Full, all rooms",B,1,1,"[https://cdn1.bazaraki.com/media/cache1/e7/ea/e7ea0ce06dd84a8c0f9dbb19e77e288d.webp, https://cdn...","1 bedroom apartment in Kato Polemidia, Limassol\nFeatures:\n• Fully furnished\n• Balcony\n• Sola...",2nd,NaN,2011,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,NaN,None,2024-12-08T19:03

In [ ]:
saledf = utils.read_last_df("output/*.parquet")

Reading last file: output/2025-11-01 00:02:19 real-estate-to-rent_real-estate-for-sale.parquet


In [ ]:
saledf.iloc[1].T

url                                                 https://www.bazaraki.com/adv/6065491_2-bedroom-detached-house-to-rent/
title                                                                                      2-bedroom semi-detached to rent
price                                                                                                                 1200
original_price                                                                                                   1300.0000
price_per_sqm                                                                                                          NaN
location                                                                                                 Larnaca, Dekeleia
posted                                                                                                    29.10.2025 20:17
ad_id                                                                                                              6065491
reference_number

In [ ]:
saledf = utils.enrich(saledf)
saledf = utils.filter_in(saledf, "`Property area` > 10 and `Property area` < 300 and price_per_sqm < 10000")
saledf = utils.filter_in(saledf, "cat1 == 'Apartments, flats for sale' and (Condition == 'Brand new' or Condition == 'Resale' or Condition.isna()) and Bedrooms in ['1', '2', 'Studio']")
saledf = utils.filter_in(saledf, "price > 150000 and price < 300000 and city == 'Larnaca'")
saledf.head()

removing 11413/39038 rows
removing 24472/27625 rows
removing 2773/3153 rows


,url,title,price,original_price,price_per_sqm,location,posted,ad_id,reference_number,views,lat,lng,sold,posted_dt,cat0,cat1,Reference number,Property area,Type,Online viewing,Energy Efficiency,Square meter price,images,description,Pets,Parking,Furnishing,Included,Air conditioning,Bedrooms,Bathrooms,Floor,Construction year,Plot area,Postal code,Minimum stay,Share,Density,Coverage,Land type,Plot Type,Parcel number,Planning zone,Registration block,Registration number,Condition,Area,city,district
8143,https://www.bazaraki.com/adv/5959854_2-bedroom-penthouse-for-sale/,2-bedroom penthouse fоr sаle,180000,NaN,2195.1220,"Larnaca, Kiti",30.10.2025 07:37,5959854,NaN,918,NaN,NaN,False,2025-10-30T07:37:00,Cyprus real estate for sale,"Apartments, flats for sale",None,82.0000,Penthouse,No,B,€2.195 /,"[https://cdn1.bazaraki.com/media/cache1/e9/62/e9627377565c3e5496b19678883dc29d.webp, https://cdn...",Fantastic 2 bedrooms Penthouse with 2 huge verandas with communal pool.\nYear of build: 2009\nIn...,None,Covered,Fully Furnished,"Pool, Garden, Elevator, Balcony","Full, all rooms",2,2,2nd,2009,NaN,7550.0000,None,None,None,None,None,None,None,None,NaN,NaN,Resale,NaN,Larnaca,Kiti
8171,https://www.bazaraki.com/adv/6053600_1-bedroom-apartment-for-sale/,1-bedroom apartment fоr sаle,250000,NaN,4901.9608,"Larnaca, Larnaka - Skala",22.10.2025 09:38,6053600,NaN,103,NaN,NaN,False,2025-10-22T09:38:00,Cyprus real estate for sale,"Apartments, flats for sale",None,51.0000,Apartment,No,A,€4.902 /,"[https://cdn1.bazaraki.com/media/cache1/b4/5b/b45ba4a823f7edf12dd839c2d91426ba.webp, https://cdn...",Park Avenue is the first development / construction project that Edge Development Group has unde...,None,Covered,Fully Furnished,"Garden, Balcony","Full, all rooms",1,1,5th,2025,NaN,6063.0000,None,None,None,None,None,None,None,None,NaN,NaN,Brand new,NaN,Larnaca,Larnaka - Skala
8525,https://www.bazaraki.com/adv/6068665_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,220000,NaN,2972.9730,"Larnaca, Larnaka - Sotiros",Yesterday 20:31,6068665,NaN,5,NaN,NaN,False,2025-10-31T20:31:16,Cyprus real estate for sale,"Apartments, flats for sale",7700,74.0000,Apartment,No,N/A,€2.973 /,"[https://cdn1.bazaraki.com/media/cache1/25/d5/25d5d84e94a3ce54f0e45be719e8b521.webp, https://cdn...","For sale is a comfortable resale apartment offering 74 square meters of internal space, thoughtf...",None,Covered,Appliances οnly,"Storage room, Elevator, Balcony","Full, all rooms",2,1,3rd,2013,NaN,NaN,None,None,None,None,None,None,None,None,NaN,NaN,Resale,NaN,Larnaca,Larnaka - Sotiros
8563,https://www.bazaraki.com/adv/6068491_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,168000,NaN,1200.0000,"Larnaca, Larnaka - Sotiros",Yesterday 20:37,6068491,NaN,15,NaN,NaN,False,2025-10-31T20:37:16,Cyprus real estate for sale,"Apartments, flats for sale",2652,140.0000,Apartment,None,In Progress,€1.200 /,"[https://cdn1.bazaraki.com/media/cache1/b3/8e/b38e9bcf91a22ad7c13cbbb79b804b90.webp, https://cdn...",Two-Bedroom Apartment for Sale in Larnaca – This asset is a spacious two-bedroom apartment locat...,None,Uncovered,None,None,"Full, all rooms",2,2,None,None,NaN,6057.0000,None,None,None,None,None,None,None,None,NaN,NaN,Resale,NaN,Larnaca,Larnaka - Sotiros
8601,https://www.bazaraki.com/adv/6021990_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,250000,NaN,2500.0000,"Larnaca, Larnaka - Harbor",Yesterday 20:07,6021990,NaN,0,NaN,NaN,False,2025-10-31T20:07:16,Cyprus real estate for sale,"Apartments, flats for sale",7258,100.0000,Apartment,No,N/A,€2.500 /,"[https://cdn1.bazaraki.com/media/cache1/f2/ad/f2adaca64f8ffc4f514dfb4096ff5f9c.webp, https://cdn...",For sale: Spacious 2-bedroom apartment in the vibrant Port area. With a generous internal space ...,None,Covered,Fully Furnished,"Storage room, Elevator, Balcony","Full, all rooms",2,2,2nd,2018,NaN,NaN,None,None,None,None,None,None,None,None,NaN,NaN,Resale,NaN,Larnaca,Larnaka - Harbor


In [ ]:

import math  


def haversine(lat1, lon1, lat2, lon2):  
    """  
    Calculate the great-circle distance between two points on the Earth (specified in decimal degrees).  
    Returns the distance in kilometers.  
    """  
    # Convert decimal degrees to radians  
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])  
    # Haversine formula  
    dlat = lat2 - lat1  
    dlon = lon2 - lon1  
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2  
    c = 2 * math.asin(math.sqrt(a))  
    # Radius of Earth in kilometers (mean radius)  
    r = 6371.0  
    return c * r  

lat1, lon1 = 52.5200, 13.4050  # Berlin  
lat2, lon2 = 52.5205, 13.4095  # Nearby point in Berlin  
print(haversine(lat1, lon1, lat2, lon2))


0.3095045367196025


In [ ]:

def rent_filter(rentdf, radius_km, sale_row):
    return rentdf[rentdf.apply(lambda r: haversine(r.lat, r.lng, sale_row.lat, sale_row.lng), axis=1) < radius_km]

def reduce_count(df, sale_row):
    if len(df) < 10:
        return df

    new_df = df[df.Bedrooms == sale_row.Bedrooms]
    if len(new_df) < 3:
        return df
    else:
        return new_df
        
def rent_finder(rentdf, radius_km, verbose, sale_row):
    if verbose:
        print(sale_row.url)
    # Step 1: Calculate the bounding box
    lat_diff = radius_km / 111.32  # Approx. 1 degree latitude = 111.32 km
    lng_diff = radius_km / (111.32 * np.cos(np.radians(sale_row.lat)))  # Adjust for latitude

    # Bounding box
    min_lat = sale_row.lat - lat_diff
    max_lat = sale_row.lat + lat_diff
    min_lng = sale_row.lng - lng_diff
    max_lng = sale_row.lng + lng_diff
    
    # Step 2: Filter rows within the bounding box
    filtered_df = rentdf[(rentdf['lat'] >= min_lat) & (rentdf['lat'] <= max_lat) & (rentdf['lng'] >= min_lng) & (rentdf['lng'] <= max_lng)]    
    filtered = rent_filter(rentdf=filtered_df, radius_km=radius_km, sale_row=sale_row)
    reduced_filtered = reduce_count(df=filtered, sale_row=sale_row)
    filtered = reduced_filtered.sort_values("delete_date", ascending=False, na_position="first")
    min_price_per_sqm = filtered.price_per_sqm.min()
    mean_price_per_sqm = filtered.price_per_sqm.median()
    avg_price_per_sqm = filtered.price_per_sqm.mean()
    max_price_per_sqm = filtered.price_per_sqm.max()
    count = filtered.price_per_sqm.count()
    mean_yield = mean_price_per_sqm * 12 / sale_row.price_per_sqm * 100
    lines = [f"{sale_row.url} {sale_row.city} price={sale_row.price} area={sale_row['Property area']} price_per_sqm={int(sale_row.price_per_sqm)} mean_yield={mean_yield:.2f} " \
                f"max_price_per_sqm:{max_price_per_sqm:.2f} min_price_per_sqm={min_price_per_sqm:.2f} avg_price_per_sqm={avg_price_per_sqm:.2f} " \
                f"mean_price_per_sqm={mean_price_per_sqm:.2f} count={count}"]
    for index, rent_row in list(filtered.iterrows())[:10]:
        lines.append(f"\t{rent_row.url} price={int(rent_row.price)} price_per_sqm={int(rent_row.price_per_sqm)}")
    return dict(mean_yield=mean_yield, text="\n".join(lines))


report_df = saledf.apply(lambda r: rent_finder(rentdf=rentdf, radius_km=1, sale_row=r, verbose=True), axis=1, result_type='expand')
report_df   

https://www.bazaraki.com/adv/5959854_2-bedroom-penthouse-for-sale/
0
https://www.bazaraki.com/adv/6053600_1-bedroom-apartment-for-sale/
0
https://www.bazaraki.com/adv/6068665_2-bedroom-apartment-for-sale/
0
https://www.bazaraki.com/adv/6068491_2-bedroom-apartment-for-sale/
0
https://www.bazaraki.com/adv/6021990_2-bedroom-apartment-for-sale/
0
https://www.bazaraki.com/adv/6022003_2-bedroom-apartment-for-sale/
0
https://www.bazaraki.com/adv/5885363_2-bedroom-apartment-for-sale/
0
https://www.bazaraki.com/adv/6021204_2-bedroom-apartment-for-sale/
0
https://www.bazaraki.com/adv/6021206_2-bedroom-apartment-for-sale/
0
https://www.bazaraki.com/adv/6021209_2-bedroom-apartment-for-sale/
0


,mean_yield,text
8143,NaN,https://www.bazaraki.com/adv/5959854_2-bedroom-penthouse-for-sale/ Larnaca price=180000 area=82....
8171,NaN,https://www.bazaraki.com/adv/6053600_1-bedroom-apartment-for-sale/ Larnaca price=250000 area=51....
8525,NaN,https://www.bazaraki.com/adv/6068665_2-bedroom-apartment-for-sale/ Larnaca price=220000 area=74....
8563,NaN,https://www.bazaraki.com/adv/6068491_2-bedroom-apartment-for-sale/ Larnaca price=168000 area=140...
8601,NaN,https://www.bazaraki.com/adv/6021990_2-bedroom-apartment-for-sale/ Larnaca price=250000 area=100...
8619,NaN,https://www.bazaraki.com/adv/6022003_2-bedroom-apartment-for-sale/ Larnaca price=295000 area=128...
8811,NaN,https://www.bazaraki.com/adv/5885363_2-bedroom-apartment-for-sale/ Larnaca price=275000 area=82....
8835,NaN,https://www.bazaraki.com/adv/6021204_2-bedroom-apartment-for-sale/ Larnaca price=189000 area=55....
8839,NaN,https://www.bazaraki.com/adv/6021206_2-bedroom-apartment-for-sale/ Larnaca price=220000 area=70....
8842,NaN,https://www.bazaraki.com/adv/6021209_2-bedroom-apartment-for-sale/ Larnaca price=159000 area=60....


In [ ]:
now = datetime.now().isoformat(sep=" ", timespec="seconds")
file_path = Path(f"output/reports/{now}_report_larnaca_150_300.txt")
file_path.parent.mkdir(parents=True, exist_ok=True)
with file_path.open("w") as fp:
    for _, row in report_df.sort_values("mean_yield", ascending=False).iterrows():
        fp.write(row.text + "\n")
        

In [ ]:
raise SystemExit

SystemExit: 

/home/ubuntu/bazaraki/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



In [ ]:
saledf.loc[5921972]

KeyError: 5921972

In [ ]:
# ENRICHING
def enrich(df):
    resultdf = saledf.parallel_apply(lambda r: rent_finder(rentdf=rentdf, radius_km=1, sale_row=r, verbose=False), axis=1, result_type='expand')
    return pd.concat([df, resultdf], axis=1)

enriched = enrich(saledf)
enriched

,url,title,price,original_price,price_per_sqm,location,posted,reference_number,views,lat,lng,sold,cat0,cat1,Property area,Pets,Type,Parking,Plot area,Furnishing,Included,Online viewing,Air conditioning,Energy Efficiency,Bedrooms,Bathrooms,images,description,Floor,Postal code,Construction year,Reference number,Condition,Square meter price,Minimum stay,Land type,Plot Type,Parcel number,Planning zone,Registration number,Share,Density,Coverage,Registration block,Area,Pick a point,posted_dt,delete_date,city,district,mean_yield,text
ad_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5774486,https://www.bazaraki.com/adv/5774486_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,220000.0000,NaN,2894.7368,"Larnaca, Oroklini",Yesterday 10:27,NaN,9,34.9841,33.6571,False,Cyprus real estate for sale,"Apartments, flats for sale",76.0000,None,Apartment,None,NaN,None,None,None,None,N/A,2,1,"[https://cdn1.bazaraki.com/media/cache1/a5/ed/a5edcdb22c4b712b3ef346b250eded41.webp, https://cdn...",Πωλείται ανακαινισμένο ισόγειο διαμέρισμα δύο υπνοδωματίων στην Ορόκλινη – επαρχία Λάρνακας με 7...,None,NaN,None,002187667,Resale,€2.895 /,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,2025-04-30T10:27:00,NaN,Larnaca,Oroklini,4.3893,https://www.bazaraki.com/adv/5774486_2-bedroom-apartment-for-sale/ Larnaca price=220000.0 area=7...
5796838,https://www.bazaraki.com/adv/5796838_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,190500.0000,NaN,2381.2500,"Larnaca, Aradippou - Agios Fanourios",13.05.2025 13:59,NaN,8,34.9437,33.5861,False,Cyprus real estate for sale,"Apartments, flats for sale",80.0000,None,Apartment,Covered,NaN,None,"Storage room, Elevator, Balcony",No,"Full, all rooms",A,2,2,"[https://cdn1.bazaraki.com/media/cache1/65/b6/65b61b265de4984bd14099e20c68daee.webp, https://cdn...","Two Bedroom Apartment For Sale In Aradippou, Larnaka\nThis beautiful apartment will have 80m2 in...",1st,NaN,None,55429,Brand new,€2.381 /,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,2025-05-13T13:59:00,NaN,Larnaca,Aradippou - Agios Fanourios,5.4161,https://www.bazaraki.com/adv/5796838_2-bedroom-apartment-for-sale/ Larnaca price=190500.0 area=8...
5798639,https://www.bazaraki.com/adv/5798639_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,225000.0000,NaN,3750.0000,"Larnaca, Larnaka - Finikoudes",14.05.2025 12:35,NaN,16,34.9156,33.6309,False,Cyprus real estate for sale,"Apartments, flats for sale",60.0000,None,Apartment,No,NaN,Fully Furnished,Balcony,No,"Full, all rooms",C,2,1,"[https://cdn1.bazaraki.com/media/cache1/36/fc/36fc421b3169e0b212267842d6e01db5.webp, https://cdn...","First Floor, Two-Bedroom Apartment For Sale in the Heart of Larnaca City Center. Within walking ...",1st,6021.0000,2006,45547,Resale,€3.750 /,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,2025-05-14T12:35:00,NaN,Larnaca,Larnaka - Finikoudes,4.3810,https://www.bazaraki.com/adv/5798639_2-bedroom-apartment-for-sale/ Larnaca price=225000.0 area=6...
5818310,https://www.bazaraki.com/adv/5818310_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,190000.0000,NaN,2968.7500,"Larnaca, Larnaka - Makenzy",26.05.2025 18:09,NaN,38,34.8999,33.6349,False,Cyprus real estate for sale,"Apartments, flats for sale",64.0000,None,Apartment,Uncovered,NaN,Fully Furnished,"Balcony, Storage room",Yes,"Full, all rooms",N/A,2,1,"[https://cdn1.bazaraki.com/media/cache1/4f/d7/4fd7f6019b26f2027c4115bbcaa9564a.webp, https://cdn...","Mackenzie area, 300 meters from 3 blue flag beaches, cafes, restaurants, 15 minutes walk to the ...",1st,6027.0000,2001,None,Resale,€2.969 /,None,None,None,None,None,6.0000,None,None,None,6.0000,NaN,NaN,2025-05-26T18:09:00,NaN,Larnaca,Larnaka - Makenzy,6.4674,https://www.bazaraki.com/adv/5818310_2-bedroom-apartment-for-sale/ Larnaca price=190000.0 area=6...
5825193,https://www.bazaraki.com/adv/5825193_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,159000.0000,NaN,2148.6486,"Larnaca, Oroklini",Yesterday 18:53,NaN,13,NaN

In [ ]:
def report(fp, df, rentdf):
    for index, row in tqdm(enriched.sort_values("mean_yield", ascending=False).iterrows()):
        row_str = f"{row.url} {row.city} price={row.price} area={row['Property area']} price_per_sqm={int(row['price_per_sqm'])} mean_yield={row['mean_yield']:.2f} " \
                  f"max_price_per_sqm:{row.max_price_per_sqm:.2f} min_price_per_sqm={row.min_price_per_sqm:.2f} avg_price_per_sqm={row.avg_price_per_sqm:.2f} " \
                  f"mean_price_per_sqm={row.mean_price_per_sqm:.2f} count={row['count']}"
        print(row_str, file=fp)
        for url in row.urls[:10]:
            result = rentdf[rentdf.url == url]
            if not result.empty:
                print(f"\t{url} price={int(result.price)} price_per_sqm={int(result.price_per_sqm)}", file=fp)

now = datetime.now().isoformat(sep=" ", timespec="seconds")
file_path = Path(f"output/reports/{now}_report.txt")
file_path.parent.mkdir(parents=True, exist_ok=True)
with file_path.open("w") as fp:
    report(fp, df=enriched, rentdf=rentdf)

0it [00:00, ?it/s]


AttributeError: 'Series' object has no attribute 'max_price_per_sqm'

In [ ]:
enriched

,url,title,price,original_price,price_per_sqm,location,posted,reference_number,views,lat,lng,sold,cat0,cat1,Property area,Pets,Type,Parking,Plot area,Furnishing,Included,Online viewing,Air conditioning,Energy Efficiency,Bedrooms,Bathrooms,images,description,Floor,Postal code,Construction year,Reference number,Condition,Square meter price,Minimum stay,Land type,Plot Type,Parcel number,Planning zone,Registration number,Share,Density,Coverage,Registration block,Area,Pick a point,posted_dt,delete_date,city,district,mean_yield,text
ad_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5367487,https://www.bazaraki.com/adv/5367487_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,380000.0000,450000.0000,5066.6667,"Limassol, Germasogeia Tourist Area",Yesterday 18:37,NaN,9,34.6954,33.0903,False,Cyprus real estate for sale,"Apartments, flats for sale",75.0000,None,Apartment,Covered,NaN,Fully Furnished,"Pool, Garden, Elevator, Balcony",None,"Full, all rooms",N/A,2,1,"[https://cdn1.bazaraki.com/media/cache1/74/15/7415a88faa79267eef118a4ad9db8bf9.webp, https://cdn...",Reduced For A Quick Sale!\nNewly Renovated Two bedroom apartment on the first line in the Germas...,2nd,NaN,None,None,Resale,€5.067 /,None,None,None,None,None,NaN,None,None,None,NaN,NaN,None,2024-12-11T18:37:00,NaN,Limassol,Germasogeia Tourist Area,5.6094,https://www.bazaraki.com/adv/5367487_2-bedroom-apartment-for-sale/ Limassol price=380000.0 area=...
5590236,https://www.bazaraki.com/adv/5590236_2-bedroom-penthouse-for-sale/,2-bedroom penthouse fоr sаle,370000.0000,NaN,4625.0000,"Larnaca, Larnaka - Agios Nikolaos",Yesterday 10:49,NaN,9,34.9125,33.6180,False,Cyprus real estate for sale,"Apartments, flats for sale",80.0000,None,Penthouse,Covered,NaN,Semi-Furnished,"Garden, Elevator, Alarm, Balcony, Storage room",No,"Full, all rooms",N/A,2,2,"[https://cdn1.bazaraki.com/media/cache1/7c/d1/7cd1e9ae2d6e2eade7ea1f73c23c6b60.webp, https://cdn...",penthouse for sale in central Larnaca Featuring salt lake views and mountains and all year aroun...,5th,6030.0000,2023,None,Resale,€4.625 /,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,2025-01-01T10:49:00,NaN,Larnaca,Larnaka - Agios Nikolaos,3.1201,https://www.bazaraki.com/adv/5590236_2-bedroom-penthouse-for-sale/ Larnaca price=370000.0 area=8...
5666162,https://www.bazaraki.com/adv/5666162_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,515000.0000,NaN,4904.7619,"Limassol, Germasogeia Tourist Area",Yesterday 14:30,NaN,9,34.6984,33.0965,False,Cyprus real estate for sale,"Apartments, flats for sale",105.0000,None,Apartment,Uncovered,NaN,Fully Furnished,"Pool, Garden, Elevator, Balcony",Yes,"Full, all rooms",B,2,1,"[https://cdn1.bazaraki.com/media/cache1/4c/5e/4c5eff249cb107d134932305a2ac57a9.webp, https://cdn...",Seafront Two Bedroom Apartment in Germasogeia Tourist Area of Limassol.\nFeatures:\n105m² Total ...,2nd,NaN,1995,None,Resale,€4.905 /,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,2025-02-21T14:30:00,NaN,Limassol,Germasogeia Tourist Area,5.7261,https://www.bazaraki.com/adv/5666162_2-bedroom-apartment-for-sale/ Limassol price=515000.0 area=...
5693595,https://www.bazaraki.com/adv/5693595_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,290000.0000,NaN,2900.0000,"Limassol, Germasogeia",10.03.2025 22:38,NaN,118,34.7149,33.0860,False,Cyprus real estate for sale,"Apartments, flats for sale",100.0000,None,Apartment,Covered,NaN,Fully Furnished,"Garden, Elevator, Balcony, Storage room",No,"Full, all rooms",A,2,1,"[https://cdn1.bazaraki.com/media/cache1/13/53/13539c744bc791310f621451c1ce5ecb.webp, https://cdn...",Modern and spacious 2 bedroom apartment located in the Primary area of Germasogia tourist Area i...,1st,4044.0000,2020,None,Resale,€2.900 /,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,2025-03-10T22:38:00,NaN,Limassol,Germasogeia,8.2759,https://www.bazaraki.com/adv/5693595_2-bedroom-apartment-for-sale/ Limassol price=290000.0 area=...
5712665,https://www.bazaraki.com

In [ ]:
enriched.sort_values("yield", price_per_sqm=False).head(20)

TypeError: DataFrame.sort_values() got an unexpected keyword argument 'price_per_sqm'